# KoBERT finetuning

In [27]:
# !pip install ipywidgets  # for vscode
# # !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [3]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
## CPU
device = torch.device("cpu")

## GPU
# device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /home/bax/문서/20221212test/.cache/kobert_v1.zip
using cached model. /home/bax/문서/20221212test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
# !wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
# !wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

# dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [34]:
# 첫번째 파라미터는 본인이 적용하고 싶은 데이터의 파일명이다.(당연하게도, tsv 파일이여야 한다)
# 두번째 파라미터 field_indices=[1,2]는 다음을 의미한다. [학습시킬 데이터의 인덱스, 데이터 레이블의 인덱스]
# 세번째 파라미터 num_discard_samples는 데이터의 상단에서 제외할 row의 개수를 의미한다. default 값은 0이다.
dataset_test = nlp.data.TSVDataset("./data2/che_test.txt", field_indices=[2,3], num_discard_samples=1)
dataset_train = nlp.data.TSVDataset("./data2/che_train.txt", field_indices=[2,3], num_discard_samples=1)

In [35]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/bax/문서/20221212test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [36]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [37]:
## Setting parameters
max_len = 256
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [38]:
# BERTDataset 클래스 이용, TensorDataset으로 만들어주기 0번째 index에 sentence가 있고 1번째 index에 label값이 있다.
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [39]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [40]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [41]:
# Bert 분류모델 생성
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [42]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [43]:
# 최적화 함수
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/home/bax/venv/mx/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [45]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [46]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [8]:
# TensorBoard 라이브러리 불러오기
from torch.utils.tensorboard import SummaryWriter

# 기본 `log_dir` 은 "runs"이며, 여기서는 더 구체적으로 지정하였습니다
writer = SummaryWriter('runs/che_log')

In [48]:
# 모델 학습하기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
    
    # 학습 로그 기록하기
    writer.add_scalar("Loss/train", loss, e+1)
    writer.add_scalar("acc/train", train_acc, e+1)
    writer.add_scalar("acc/test", test_acc, e+1)
    writer.flush()
writer.close()



  0%|          | 0/3 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1275441646575928 train acc 0.28125
epoch 1 train acc 0.37202380952380953


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1 test acc 0.5897435897435898


  0%|          | 0/3 [00:00<?, ?it/s]

...

epoch 19 test acc 0.6153846153846154


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.5807570815086365 train acc 0.828125
epoch 20 train acc 0.8720238095238096


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 20 test acc 0.6153846153846154


In [9]:
# 터미널에서 실행
tensorboard --logdir=runs

SyntaxError: cannot assign to operator (1405008375.py, line 2)